# HW2

This homework will explore reading in raster data, band algebra, masking rasters, and aggregating over polygons. Thematically, this homework will be about measuring the total amount of cropland planted with different crops in the United States using the Cropland Data Layer, a data product produced by USDA that maps predicted crop type at 30m resolution.

The first step is to download the data [here](https://www.nass.usda.gov/Research_and_Science/Cropland/Release/datasets/2017_30m_cdls.zip) (*Note:* this is a big download! It's 1.7 GB zipped and almost 8 GB unzipped, so make sure you have space on your harddrive.) Once the download has finished, unzip it and feel free to move it into this directory for easy access. In any case, note the path where it's saved, because the first step will be to read it in.

In [ ]:
import rasterio as rio

cdl = # Open the CDL file ending in "".tif"
cdl

That may have been surprisingly fast. That's because `rasterio` doesn't actually load any of the data into memory, just the metadata (until we try using the `read` method).

First, let's check what CRS the raster uses.

In [ ]:
# Check the CRS

Let's also check how many bands it has.

In [ ]:
# Check the number of bands

Next, let's get a shapefile of counties in the United States from [here](https://www2.census.gov/geo/tiger/TIGER2017/COUNTY/tl_2017_us_county.zip). As usual, unzip it and feel free to move it around. We'll read this one in next and plot it.

In [ ]:
import geopandas as gpd

# Read in the county shapefile (ending in ".shp")
counties = 
counties.plot() # plot it

We're only interested in the continental US for this homework. Try making a box that will filter to those counties. (Note that by default, geopandas assigns the name "geometry" to the column containing the spatial information).

In [ ]:
import shapely.geometry as shg

box = shg.box() # Choose coordinates of a box that contains the continental US
# apply a function that will check that the county intersects your box
continental = counties.loc[counties['geometry'].apply(lambda x:)] 
continental.plot()

Now we've got our counties. Let's transorm them to the same CRS as the CDL so we can mask things without issue.

In [ ]:
continental = continental. # Transform it to the CDL's crs
continental.plot() # plot to make sure units are different.

Now let's do an example of masking using Dane county! First, we need to get a sense of what the columns in the county dataset are so we can figure out how we can get the geometry for Dane county.

In [ ]:
continental

Now try getting the geometry for Dane county (here I've given you a little help by showing how you need to use `iloc` to actually grab the value in the DataFrame).

In [ ]:
dane = continental.loc[].iloc[0]
dane

Now let's mask the CDL and see what's growing in Dane county! This will be pretty similar to the masking exercise we did in class, but using real data.

In [ ]:
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np

# Remember we use _ to hold the transform part of the output from mask
dane_cdl,_ = mask(,crop=True) # Don't forget crop=True unless you want to try to load the entire 8GB raster!
plt.imshow(dane_cdl[0,:,:]) # Output adds a first band dimension

Neat! You should be able to see Madison and the lakes pretty clearly. Unfortunately, these colors aren't really interpretable. If you look in the html file that came with the CDL data, you'll see that different values are associated with different crops. Let's try plotting all the corn using some band algebra.

In [ ]:
# Look up the value associated with corn and make a new array that is 1 for corn and 0 otherwise
corn = 
plt.imshow(corn[0,:,:])

That's a lot of corn! Let's see exactly how much it is by aggregating up to the county.

In [ ]:
# Sum up the number of corn pixels and convert to hectares
# each pixel is 30 m x 30 m and a hectare is 100 m x 100 m

np.sum(corn)*(30**2)/(100**2)

Now let's make a general function that will count the hectares of any crop in any county based on the example we did above.

In [ ]:
def crop_ha(crop,county):
    county_cdl,_ = mask(,crop=True) # This should be similar to the Dane county example
    county_crop =  # Similar
    return(np.sum(county_crop)*(30**2)/(100**2))

crop_ha(,dane) # test it on corn in Dane county

In [ ]:
crop_ha(,dane) # now try it with soybeans

Now let's put this all together to make some maps of total area dedicated to different crops. We'll apply our new function to the counties, iterating over different crops. Then we'll plot the results. This one will take a while!

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(20,20))

crops = []# do corn, soybeans, cotton, and winter wheat
# Let's also make a dictionary to convert the codes associated with each crop to their name
name_dict = {:'corn',
            :'soy',
            :'cotton',
            :'winter wheat'}

for i in range(0,4): # This will make plotting easier 
    crop = crops[i] # select the crop we'll be doing for this iteration
    continental[f'ha_{crop}'] = continental[].apply(lambda x:) # apply the crop_ha fn for that crop
    continental.plot(column=f'ha_{crop}',ax=axs[i//2,i%2], # this moves us among the subplots, doing the top then bottom row
                    legend=True)
    
    axs[i//2,i%2].set_title(name_dict[crop])